# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import re
import string
import zipfile
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras

from __future__ import print_function

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.python.keras import layers
from tensorflow.python.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.python.keras.models import Model

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Load the dataset

In [0]:
zip_ref = zipfile.ZipFile('drive/My Drive/Datasets/mar-eng.zip', 'r')
zip_ref.extractall('mar-eng') # unzip directory
zip_ref.close()

In [0]:
batch_size = 64  # Batch size for training.
epochs = 200  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'mar-eng/mar.txt'

# Vectorize the data.

In [0]:

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)



In [23]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 85
Max sequence length for inputs: 19
Max sequence length for outputs: 42


In [0]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# Encoder

In [0]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [0]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder

In [0]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [0]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Build the Model

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 85)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 335872      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  350208      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [52]:
from tensorflow.python.keras.callbacks import EarlyStopping

early_stopping_monitor = EarlyStopping(patience=3)

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,callbacks=[early_stopping_monitor],verbose=1,
          validation_split=0.2)

Epoch 1/200
125/125 [==============================] - 66s 531ms/step - loss: 0.0397 - accuracy: 0.9846 - val_loss: 1.2734 - val_accuracy: 0.8046
Epoch 2/200
125/125 [==============================] - 55s 440ms/step - loss: 0.0400 - accuracy: 0.9844 - val_loss: 1.2788 - val_accuracy: 0.8030
Epoch 3/200
125/125 [==============================] - 60s 479ms/step - loss: 0.0396 - accuracy: 0.9845 - val_loss: 1.2738 - val_accuracy: 0.8050
Epoch 4/200
125/125 [==============================] - 63s 505ms/step - loss: 0.0393 - accuracy: 0.9844 - val_loss: 1.2725 - val_accuracy: 0.8050
Epoch 5/200
125/125 [==============================] - 57s 455ms/step - loss: 0.0390 - accuracy: 0.9847 - val_loss: 1.2860 - val_accuracy: 0.8038
Epoch 6/200
125/125 [==============================] - 61s 492ms/step - loss: 0.0386 - accuracy: 0.9847 - val_loss: 1.2870 - val_accuracy: 0.8048
Epoch 7/200
125/125 [==============================] - 61s 490ms/step - loss: 0.0385 - accuracy: 0.9846 - val_loss: 1.2842 -

In [0]:
model.save('s2s_char.h5')

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [0]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

# Predictions

In [57]:
def decode_sequence(input_seq):
    
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
 
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

       
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: जा.

-
Input sentence: Run!
Decoded sentence: पळा!

-
Input sentence: Run!
Decoded sentence: पळा!

-
Input sentence: Run!
Decoded sentence: पळा!

-
Input sentence: Run!
Decoded sentence: पळा!

-
Input sentence: Who?
Decoded sentence: कोण?

-
Input sentence: Wow!
Decoded sentence: वाह!

-
Input sentence: Fire!
Decoded sentence: आग!

-
Input sentence: Fire!
Decoded sentence: आग!

-
Input sentence: Help!
Decoded sentence: वाचवा!

-
Input sentence: Help!
Decoded sentence: वाचवा!

-
Input sentence: Jump!
Decoded sentence: उडी मार!

-
Input sentence: Jump!
Decoded sentence: उडी मार!

-
Input sentence: Jump.
Decoded sentence: उडी मारा.

-
Input sentence: Jump.
Decoded sentence: उडी मारा.

-
Input sentence: Stop!
Decoded sentence: थांबा!

-
Input sentence: Stop!
Decoded sentence: थांबा!

-
Input sentence: Wait!
Decoded sentence: थांब!

-
Input sentence: Wait!
Decoded sentence: थांब!

-
Input sentence: Hello!
Decoded sentence: पकड!

-
Input sentence: Hurr